# Data analysis for MCMC calculations of energies
## Problem 1b) Brute Force sampling
### Fabian Faulstich and Yngve Mardal Moe

In [1]:
# Base python imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time
from timeit import timeit
from joblib import Parallel, delayed
from collections import namedtuple

import scipy.stats as stats
from statsmodels.tsa.stattools import acovf

# VMC imports
from particles import Particles
from wave_function import WaveFunction
from sampler import MetropolisSampler, ImportanceSampler
from vmc import VMC

/home/yngve/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Supress divide by zero warnings
np.seterr('ignore')

# IPython magics
%matplotlib inline
%load_ext autoreload
%autoreload 2

We start by defining a simple function that simulate the system according to the parameters given in problem 1b)

In [3]:
def create_sampler(num_particles, num_dimensions, alpha, step_size, seed=None):
    parts = Particles(
        num_particles=num_particles, 
        num_dimensions=num_dimensions,
        mass=1,
        diameter=0
    )
    wave_function = WaveFunction(
        particles=parts,
        alpha=alpha,
        beta=1,
        omega_ho=1,
        omega_z=1
    )
    sampler = MetropolisSampler(
        wave_function=wave_function,
        step_size=step_size,
        seed=seed
    )
    return sampler


def simulate_simple(num_particles, num_dimensions, alpha, step_size,
                    num_iterations, seed=None):
    sampler = create_sampler(
        num_particles=num_particles,
        num_dimensions=num_dimensions,
        alpha=alpha,
        step_size=step_size,
        seed=seed
    )
    return sampler.compute_local_energy(num_iterations)

Result = namedtuple('Result', ['params', 'energies', 'positions'])

In [4]:
alphas = np.array([0.01, 0.1, 0.5, 0.6, 0.7, 1])
num_particles = np.array([1, 10, 100, 500])
num_dimensions = np.array([1, 2, 3])

particles_params = [
    {'num_particles': n, 'num_dimensions': d, 'mass': 1, 'diameter': 0}
        for n in num_particles for d in num_dimensions
]
wf_params = {'beta': 1, 'omega_ho': 1, 'omega_z': 1}


In [5]:
vmcs = {}
for pp in particles_params:
    n = pp['num_particles']
    d = pp['num_dimensions']
    vmcs[(n, d)] = VMC(MetropolisSampler, wf_params, pp)

In [16]:
energies = {}
variances = {}
MCMC_results = {}

num_its=int(np.exp(int(np.log2(10000))+1))
def perform_inference(vmc, params, alpha):
    import numpy as np
    np.seterr('ignore')
    energies, positions = vmc.perform_mc_iterations(alpha, num_iterations=num_its,
                                                    ideal_ar=0.5, return_positions=True)
    result = Result((alpha, *params), energies, positions)
    return result

for alpha in alphas:
    results = Parallel(n_jobs=1)(
        delayed(perform_inference)(param, vmc, alpha) for vmc, param in vmcs.items()
    )
    for result in results:
        param = result.params
        energies[param] = result.energies.mean()
        variances[param] = result.energies.var()/num_its        
        MCMC_results[param] = result
    print(f'Finished all configurations for alpha={alpha}')
    


/home/yngve/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


MemoryError: 

In [8]:
def expected_energy(parameters):
    return np.prod(parameters)/2 + parameters[1]*parameters[2]/(parameters[0]*8)

print('| Alpha |  n  | d | Computed energy |   Variances  | Number of Iterations |')
print('|-------|-----|---|-----------------|--------------|----------------------|')
for param, energy in energies.items():
    print(f'|{param[0]:6.3f} |{param[1]:4d} | {param[2]} |{energy:16.4f} | {variances[param]:12.1f} | {num_its} |')


| Alpha |  n  | d | Computed energy |   Variances  | Number of Iterations |
|-------|-----|---|-----------------|--------------|----------------------|
| 0.010 |   1 | 1 |         12.7369 |          2.4 | 17 |
| 0.010 |   1 | 2 |         13.2690 |          1.6 | 17 |
| 0.010 |   1 | 3 |         44.1344 |         22.7 | 17 |
| 0.010 |  10 | 1 |         91.4033 |         36.9 | 17 |
| 0.010 |  10 | 2 |        256.8024 |         18.4 | 17 |
| 0.010 |  10 | 3 |        531.0363 |         22.1 | 17 |
| 0.010 | 100 | 1 |       1192.7842 |          7.8 | 17 |
| 0.010 | 100 | 2 |       2416.1190 |         13.2 | 17 |
| 0.010 | 100 | 3 |       3719.2942 |        137.4 | 17 |
| 0.010 | 500 | 1 |       3607.6305 |         25.2 | 17 |
| 0.010 | 500 | 2 |       5139.8601 |         52.2 | 17 |
| 0.010 | 500 | 3 |       6525.8225 |         10.0 | 17 |
| 0.100 |   1 | 1 |          2.7446 |          0.3 | 17 |
| 0.100 |   1 | 2 |          2.9591 |          0.2 | 17 |
| 0.100 |   1 | 3 |          4.2905 

This table comes from copying the above output

| Alpha |  n  | d | Computed energy |   Variances  | Number of Iterations |
|-------|-----|---|-----------------|--------------|----------------------|
| 0.010 |   1 | 1 |         12.7369 |          2.4 | 17 |
| 0.010 |   1 | 2 |         13.2690 |          1.6 | 17 |
| 0.010 |   1 | 3 |         44.1344 |         22.7 | 17 |
| 0.010 |  10 | 1 |         91.4033 |         36.9 | 17 |
| 0.010 |  10 | 2 |        256.8024 |         18.4 | 17 |
| 0.010 |  10 | 3 |        531.0363 |         22.1 | 17 |
| 0.010 | 100 | 1 |       1192.7842 |          7.8 | 17 |
| 0.010 | 100 | 2 |       2416.1190 |         13.2 | 17 |
| 0.010 | 100 | 3 |       3719.2942 |        137.4 | 17 |
| 0.010 | 500 | 1 |       3607.6305 |         25.2 | 17 |
| 0.010 | 500 | 2 |       5139.8601 |         52.2 | 17 |
| 0.010 | 500 | 3 |       6525.8225 |         10.0 | 17 |
| 0.100 |   1 | 1 |          2.7446 |          0.3 | 17 |
| 0.100 |   1 | 2 |          2.9591 |          0.2 | 17 |
| 0.100 |   1 | 3 |          4.2905 |          0.2 | 17 |
| 0.100 |  10 | 1 |         12.4721 |          0.2 | 17 |
| 0.100 |  10 | 2 |         21.1957 |          0.4 | 17 |
| 0.100 |  10 | 3 |         38.8197 |          0.2 | 17 |
| 0.100 | 100 | 1 |        147.1127 |          0.4 | 17 |
| 0.100 | 100 | 2 |        230.4900 |          0.5 | 17 |
| 0.100 | 100 | 3 |        377.9683 |          0.2 | 17 |
| 0.100 | 500 | 1 |        499.2954 |          0.3 | 17 |
| 0.100 | 500 | 2 |        937.1640 |          0.3 | 17 |
| 0.100 | 500 | 3 |       1235.4324 |          0.8 | 17 |
| 0.500 |   1 | 1 |          0.5000 |          0.0 | 17 |
| 0.500 |   1 | 2 |          1.0000 |          0.0 | 17 |
| 0.500 |   1 | 3 |          1.5000 |          0.0 | 17 |
| 0.500 |  10 | 1 |          5.0000 |          0.0 | 17 |
| 0.500 |  10 | 2 |         10.0000 |          0.0 | 17 |
| 0.500 |  10 | 3 |         15.0000 |          0.0 | 17 |
| 0.500 | 100 | 1 |         50.0000 |          0.0 | 17 |
| 0.500 | 100 | 2 |        100.0000 |          0.0 | 17 |
| 0.500 | 100 | 3 |        150.0000 |          0.0 | 17 |
| 0.500 | 500 | 1 |        250.0000 |          0.0 | 17 |
| 0.500 | 500 | 2 |        500.0000 |          0.0 | 17 |
| 0.500 | 500 | 3 |        750.0000 |          0.0 | 17 |
| 0.600 |   1 | 1 |          0.5587 |          0.0 | 17 |
| 0.600 |   1 | 2 |          1.0953 |          0.0 | 17 |
| 0.600 |   1 | 3 |          1.5510 |          0.0 | 17 |
| 0.600 |  10 | 1 |          5.4074 |          0.0 | 17 |
| 0.600 |  10 | 2 |          9.3620 |          0.0 | 17 |
| 0.600 |  10 | 3 |         14.5817 |          0.0 | 17 |
| 0.600 | 100 | 1 |         54.5359 |          0.0 | 17 |
| 0.600 | 100 | 2 |        101.4007 |          0.0 | 17 |
| 0.600 | 100 | 3 |        142.1813 |          0.0 | 17 |
| 0.600 | 500 | 1 |        223.8976 |          0.0 | 17 |
| 0.600 | 500 | 2 |        423.0250 |          0.0 | 17 |
| 0.600 | 500 | 3 |        607.9479 |          0.0 | 17 |
| 0.700 |   1 | 1 |          0.4590 |          0.0 | 17 |
| 0.700 |   1 | 2 |          1.1365 |          0.0 | 17 |
| 0.700 |   1 | 3 |          1.6624 |          0.0 | 17 |
| 0.700 |  10 | 1 |          5.2136 |          0.0 | 17 |
| 0.700 |  10 | 2 |         10.8396 |          0.0 | 17 |
| 0.700 |  10 | 3 |         16.3129 |          0.0 | 17 |
| 0.700 | 100 | 1 |         49.9339 |          0.0 | 17 |
| 0.700 | 100 | 2 |        104.2807 |          0.0 | 17 |
| 0.700 | 100 | 3 |        142.6135 |          0.0 | 17 |
| 0.700 | 500 | 1 |        173.5229 |          0.0 | 17 |
| 0.700 | 500 | 2 |        351.5764 |          0.0 | 17 |
| 0.700 | 500 | 3 |        486.6581 |          0.0 | 17 |
| 1.000 |   1 | 1 |          0.6578 |          0.0 | 17 |
| 1.000 |   1 | 2 |          1.2179 |          0.0 | 17 |
| 1.000 |   1 | 3 |          1.9152 |          0.0 | 17 |
| 1.000 |  10 | 1 |          6.4619 |          0.0 | 17 |
| 1.000 |  10 | 2 |         16.1579 |          0.0 | 17 |
| 1.000 |  10 | 3 |         16.8131 |          0.0 | 17 |
| 1.000 | 100 | 1 |         63.4344 |          0.0 | 17 |
| 1.000 | 100 | 2 |        105.9830 |          0.0 | 17 |
| 1.000 | 100 | 3 |         74.4563 |          0.1 | 17 |
| 1.000 | 500 | 1 |        107.5886 |          0.0 | 17 |
| 1.000 | 500 | 2 |        -55.9417 |          0.0 | 17 |
| 1.000 | 500 | 3 |       -420.6537 |          0.5 | 17 |



Thus, we see that our MCMC sampler does indeed seem to give the correct result for the non-interacting case.

----------------------
## Timings
Let us now see what the complexity of the MCMC sampler is. Here, $m$ denote the number of iterations, $n$ the number of particles and $d$ the number of dimensions.

We choose alpha equal to 0.5 for these tests.

The computational complexity scales linearly with the number of iterations, $m$, as each iteration does the exact same thing. We therefore only need to test the computational complexity wrt $n$ and $d$.

In [9]:
alpha = 0.5
num_particles = np.logspace(0, 2, 10, dtype=int)
num_dimensions = np.array([1, 2, 3])
speed_samplers = {}

for n in num_particles:
    for d in num_dimensions:
        speed_samplers[(n, d)] = create_sampler(n, d, alpha, 0.1)

In [11]:
for sampler in speed_samplers.values():
    sampler.find_ideal_step_size(0.01, num_iterations=1000)

/home/yngve/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [12]:
timings = {}
num_it = 100
for params, sampler in speed_samplers.items():
    print(params)
    timings[params] = timeit(lambda : sampler.compute_local_energy(num_it),
                             number=100)
    

(1, 1)
(1, 2)
(1, 3)
(2, 1)
(2, 2)
(2, 3)
(4, 1)
(4, 2)
(4, 3)
(7, 1)
(7, 2)
(7, 3)
(12, 1)
(12, 2)
(12, 3)
(21, 1)
(21, 2)
(21, 3)


KeyboardInterrupt: 

In [ ]:
plt.figure()
for d in num_dimensions:
    runtime = [timings[(n, d)]/num_it for n in num_particles]
    plt.plot(num_particles, runtime)

plt.legend([f'{d} dimensions' for d in num_dimensions])
plt.title('Runtime per iteration')
plt.xlabel('Number of particles')
plt.ylabel('Time [s]')
plt.show()

This seem like it might be squared complexity. Let us therefore perform a square root transform.

In [ ]:
plt.figure()
for d in num_dimensions:
    runtime = np.array([timings[(n, d)]/num_it for n in num_particles])
    plt.plot(num_particles, np.sqrt(runtime))
    plt.title('Root runtime as a function of number of particles')

plt.legend([f'{d} dimensions' for d in num_dimensions])
plt.title('Runtime per iteration')
plt.xlabel('Number of particles')
plt.ylabel('Time [s]')
plt.show()

From this plot, it seems like there is a quadratic complexity with respect to the number of parameters (in the non-interacting case that is. In the interacting case there is a tripple loop which is skipped if the Boson diameter is 0).

## Advanced statistical analysis

Now, we want to perform some advanced statistical analysis on our results. Mainly, to compute the variances using the blocking method, which gives a more realistic estimate.

In [13]:
def blocking(energies):
    energies = np.array(energies)
    d = int(np.log2(len(energies)))
    energies = energies[:2**d]
    mean = energies.mean()
    
    autocovariances = np.zeros(d)
    variances = np.zeros(d)
    block_sizes = 2**(np.arange(d)) 
    
    for i in range(d):
        n = len(energies)
        autocovariances[i] = np.sum(
            (energies[:-1] - mean)*(energies[1:] - mean)
        )/(n-1)
        variances[i] = np.var(energies, ddof=1)
        
        energies = (energies[::2] + energies[1::2])/2
    
    autocorrelations = autocovariances/variances
    chi2 = np.cumsum((2*block_sizes[::-1]*autocorrelations**2)[::-1])[::-1]
        # equivalent to sum(...) - cumsum(...)
    p_value = stats.chi2.cdf(chi2, df=np.arange(d)+1)
    variances /= 2**(d-np.arange(d))
    
    index = pd.Series(block_sizes, name='Block size')
    result = {
        'Autocovariance': autocovariances,
        'Variance': variances,
        'Autocorrelations': autocorrelations,
        '$\chi$': chi2,
        'p_value': p_value
    }
    return pd.DataFrame(result, index=index)

def realistic_variance(energies, significance_level=0.01):
    blocking_df = blocking(energies)
    try:
        return blocking_df.Variance[blocking_df.p_value < 1-significance_level].values[0]
    except IndexError:
        return np.nan

In [14]:
result.energies

array([-425.50707043, -425.50707043, -422.39032859, -422.39032859,
       -422.39032859, -422.78778373, -421.94823482, -421.56945181,
       -421.56945181, -420.02293407, -420.02293407, -420.02293407,
       -419.14543493, -416.41126514, -416.41126514, -416.5076688 ,
       -416.5076688 ])

In [15]:
print('| Alpha |  n  | d | Computed energy | Naive variance | Actual variance |')
print('|-------|-----|---|-----------------|----------------|--------------------|')
for param, result in MCMC_results.items():
    energy = result.energies.mean()
    n_var = result.energies.var(ddof=1)/num_its
    a_var = realistic_variance(result.energies)
    print(f'|{param[0]:6.3f} |{param[1]:4d} | {param[2]} |{energy:16.4f} | {n_var:12.1f} | {a_var:12.1f} |')
    

| Alpha |  n  | d | Computed energy | Naive variance | Actual variance |
|-------|-----|---|-----------------|----------------|--------------------|
| 0.010 |   1 | 1 |         12.7369 |          2.6 |          2.9 |
| 0.010 |   1 | 2 |         13.2690 |          1.7 |          1.2 |
| 0.010 |   1 | 3 |         44.1344 |         24.1 |         43.1 |
| 0.010 |  10 | 1 |         91.4033 |         39.2 |         83.5 |
| 0.010 |  10 | 2 |        256.8024 |         19.6 |         36.4 |
| 0.010 |  10 | 3 |        531.0363 |         23.5 |         26.5 |
| 0.010 | 100 | 1 |       1192.7842 |          8.3 |         16.4 |
| 0.010 | 100 | 2 |       2416.1190 |         14.0 |         24.6 |
| 0.010 | 100 | 3 |       3719.2942 |        145.9 |        282.2 |
| 0.010 | 500 | 1 |       3607.6305 |         26.8 |         61.6 |
| 0.010 | 500 | 2 |       5139.8601 |         55.5 |        120.3 |
| 0.010 | 500 | 3 |       6525.8225 |         10.6 |         18.6 |
| 0.100 |   1 | 1 |          2.7446

| Alpha |  n  | d | Computed energy | Naive variance | Actual variance |
|-------|-----|---|-----------------|----------------|--------------------|
| 0.010 |   1 | 1 |          2.4113 |          7.7 |          0.5 |
| 0.010 |   1 | 2 |         17.6183 |        156.2 |          9.1 |
| 0.010 |   1 | 3 |         30.9143 |        116.1 |          7.7 |
| 0.010 |  10 | 1 |        115.1956 |        725.4 |         81.4 |
| 0.010 |  10 | 2 |        204.6076 |        819.7 |        103.4 |
| 0.010 |  10 | 3 |        408.3209 |        757.6 |         95.9 |
| 0.010 | 100 | 1 |       1600.5880 |        310.0 |         16.2 |
| 0.010 | 100 | 2 |       2419.6584 |       1549.8 |        209.2 |
| 0.010 | 100 | 3 |       3318.7094 |       1028.7 |        135.3 |
| 0.100 |   1 | 1 |          0.5427 |          0.3 |          0.0 |
| 0.100 |   1 | 2 |          2.2088 |          1.7 |          0.1 |
| 0.100 |   1 | 3 |          3.4919 |         12.8 |          0.7 |
| 0.100 |  10 | 1 |         17.4955 |          0.9 |          0.0 |
| 0.100 |  10 | 2 |         31.3913 |          6.8 |          0.8 |
| 0.100 |  10 | 3 |         29.2829 |          8.9 |          1.1 |
| 0.100 | 100 | 1 |        136.0351 |          6.0 |          0.8 |
| 0.100 | 100 | 2 |        231.6789 |          4.8 |          0.3 |
| 0.100 | 100 | 3 |        339.5157 |         15.7 |          0.8 |
| 0.500 |   1 | 1 |          0.5000 |          0.0 |          nan |
| 0.500 |   1 | 2 |          1.0000 |          0.0 |          nan |
| 0.500 |   1 | 3 |          1.5000 |          0.0 |          nan |
| 0.500 |  10 | 1 |          5.0000 |          0.0 |          nan |
| 0.500 |  10 | 2 |         10.0000 |          0.0 |          nan |
| 0.500 |  10 | 3 |         15.0000 |          0.0 |          nan |
| 0.500 | 100 | 1 |         50.0000 |          0.0 |          nan |
| 0.500 | 100 | 2 |        100.0000 |          0.0 |          nan |
| 0.500 | 100 | 3 |        150.0000 |          0.0 |          nan |
| 1.000 |   1 | 1 |          0.8419 |          0.0 |          0.0 |
| 1.000 |   1 | 2 |          1.3567 |          0.4 |          0.0 |
| 1.000 |   1 | 3 |          2.4603 |          0.1 |          0.0 |
| 1.000 |  10 | 1 |          6.9478 |          0.2 |          0.0 |
| 1.000 |  10 | 2 |         11.2608 |          0.3 |          0.0 |
| 1.000 |  10 | 3 |         21.2935 |          0.3 |          0.0 |
| 1.000 | 100 | 1 |         63.7954 |          0.3 |          0.0 |
| 1.000 | 100 | 2 |         92.0392 |          0.4 |          0.0 |
| 1.000 | 100 | 3 |         99.0496 |          0.5 |          0.1 |

The code above is benchmarked against Marius' code (proof at end of notebook) and gives the same results. Furthermore, we benchmark it against the analytically computed standard error. This is can be done easily using the discrete convolution theorem, which allows us to compute the aautocorrelation with complexity $O(nlog(n))$ [1].

[1]: Numerical Recipes in C: The Art of Scientific Computing, 3rd edition

-------------------------
## Benchmarking with Marius' code

In [ ]:
def block(x):
    # preliminaries
    n = len(x)
    d = np.log2(n).astype(int)
    s, gamma = np.zeros(d), np.zeros(d)
    mu = np.mean(x)

    # estimate the auto-covariance and variances 
    # for each blocking transformation
    for i in range(0,d):
        n = len(x)
        # estimate autocovariance of x
        gamma[i] = (n-1)**(-1)*np.sum( (x[0:(n-1)]-mu)*(x[1:n]-mu) )
        # estimate variance of x
        s[i] = np.var(x, ddof=1)
        # perform blocking transformation
        x = 0.5*(x[0::2] + x[1::2])
        
    # generate the test observator Z_k from the theorem

    Z = (np.cumsum( ((gamma/s)**2*2**np.arange(1,d+1)[::-1])[::-1] )  )[::-1]
    # we need a list of magic numbers
    q = np.array([6.634897,9.210340, 11.344867, 13.276704, 15.086272, 16.811894, 18.475307, 20.090235, 21.665994, 23.209251, 24.724970, 26.216967, 27.688250, 29.141238, 30.577914, 31.999927, 33.408664, 34.805306, 36.190869, 37.566235, 38.932173, 40.289360, 41.638398, 42.979820, 44.314105, 45.641683, 46.962942, 48.278236, 49.587884, 50.892181])

    # use magic to determine when we should have stopped blocking
    for k in np.arange(0,d):
        if(Z[k] < q[k]):
            break
    return s[k]/2**(d-k)

In [ ]:
print(' Difference between my and Marius\' implementation')
for result in MCMC_results.values():
    e = result.energies
    print('-'*50)
    print(block(e[:2**int(np.log2(len(e)))])-realistic_variance(e))

## Benchmarking with analytical solution

In [ ]:
e = MCMC_results[1, 1, 3].energies
n = len(e)
acov = acovf(e, fft=True)
k = np.arange(0, len(acov))
(((n-k)/n)*acov)[0], acov[0]

In [ ]:
def true_variance(e):
    n = len(e)
    acov = acovf(e, fft=True)
    k = np.arange(0, n)
    acov_prefix = (n-k)/n
    se = (2*np.sum((n-k)*acov/n) - acov[0])/n

    return se
    
for result in MCMC_results.values():
    e = result.energies
    print('-'*50)
    print(realistic_variance(e), true_variance(e))